In [7]:
%matplotlib inline
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import pickle
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import cross_val_score,KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import RobustScaler



import warnings


warnings.filterwarnings("ignore")

# environment settings
data_path = 'Data/'

# Deserialize previously saved data from "preprocessing"
with open(data_path+'train_pp.obj', 'rb') as train_pp, \
open(data_path+'test_pp.obj','rb') as test_pp:
    train_df = pickle.load(train_pp)
    test_df = pickle.load(test_pp)
train_df["SalePrice"] = np.log1p(train_df["SalePrice"])

In [8]:
X = train_df.loc[:,'Id':'2ndFlrSF-Sq']
y = train_df['SalePrice']
print("Shape of training set {}.\nShape of test set {}".format(X.shape,y.shape))

Shape of training set (1456, 349).
Shape of test set (1456,)


In [9]:
scaler = RobustScaler()
X_scaled=scaler.fit(X).transform(X)
test_scaled=scaler.transform(test_df)

In [11]:
gBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.02,
                                   max_depth=3, max_features='sqrt', 
                                   loss='huber', random_state =5)

kf = KFold(10, shuffle=True, random_state=42).get_n_splits(X_scaled)
rmse_cv_gB= np.sqrt(-cross_val_score(gBoost, X, y, scoring="neg_mean_squared_error", cv = kf))
print("The 10-fold crossvalidation RMSE of GBoost is {:.5f} +/- {:.3f}".format(rmse_cv_gB.mean(),
                                                                               rmse_cv_gB.std()))

The 10-fold crossvalidation RMSE of GBoost is 0.10942 +/- 0.017


The 10-fold crossvalidation RMSE of GBoost is 0.10882 +/- 0.017 <br>
gBoost = GradientBoostingRegressor(n_estimators=4000, learning_rate=0.03, <br>
                                   max_depth=2, max_features='sqrt', <br>
                                   loss='huber', random_state =5)

In [5]:
#Prediction on real test set using GBoost
gBoost.fit(X_scaled,y)
pred_results =gBoost.predict(test_scaled)
pred_results = np.expm1(pred_results)
result_df = pd.DataFrame(data={'Id': test_df["Id"].values,
                               'SalePrice': pred_results})
#Create output csv file
result_df.to_csv(data_path+"outputs/gb_3000", index=False)